In [ ]:
# AI Web Lip Syncing Application

#Author:** Abdul Salam Wahab
#University Email:** bc210200607@vu.edu.pk
#GitHub (if any):** https://github.com/AbdulSalamvu/Wav2Lip.git
#Institution:** Virtual University of Pakistan
#Final Year Project - BSc in Computer Science**

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok authtoken 2p1yhE3O26S5y6W7jAtCpR6MMdn_5uSiAYobvDwYaFMw1bdYC

In [ ]:
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/AbdulSalamvu/Wav2Lip.git

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done


In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1
!pip install python-multipart

In [ ]:
import sys
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse, JSONResponse
import os
import subprocess
from pathlib import Path
import shutil
import logging


# Initialize FastAPI app
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)


# Directories for input and output
BASE_DIR = Path("/content/Wav2Lip")
INPUT_DIR = BASE_DIR / "inputs"
OUTPUT_DIR = BASE_DIR / "results"

# Ensure directories exist
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Set directories for storing files
UPLOAD_DIR = '/content/sample_data/'
if not os.path.exists(UPLOAD_DIR):
    os.makedirs(UPLOAD_DIR)

# Configure logging for better debugging
logging.basicConfig(level=logging.DEBUG)



@app.get("/test")
def test_api():
    return JSONResponse(status_code=200, content={"message": "API is UP !"})


@app.post("/upload")
def upload_files(video: UploadFile = File(...), audio: UploadFile = File(...)):
    # Debugging: Log the incoming request data
    print(f"Received video file: {video.filename}")
    print(f"Received audio file: {audio.filename}")
    print(f"Video file content type: {video.content_type}")
    print(f"Audio file content type: {audio.content_type}")

    # Define the paths where the files will be saved
    video_path = os.path.join(UPLOAD_DIR, video.filename)
    audio_path = os.path.join(UPLOAD_DIR, audio.filename)

    try:
        # Save video file
        with open(video_path, "wb") as video_file:
            shutil.copyfileobj(video.file, video_file)

        # Save audio file
        with open(audio_path, "wb") as audio_file:
            shutil.copyfileobj(audio.file, audio_file)

        return JSONResponse(status_code=200, content={"message": "Files uploaded successfully", "video": video.filename, "audio": audio.filename})

    except Exception as e:
        # Log the error and return response
        print(f"Error during file upload: {str(e)}")
        return JSONResponse(status_code=500, content={"error": str(e)})

@app.post("/process")
def process_files(video_filename: str, audio_filename: str):
    """
    Process the uploaded files using Wav2Lip.
    """
    video_path = INPUT_DIR / video_filename
    audio_path = INPUT_DIR / audio_filename
    output_path = OUTPUT_DIR / "result_voice.mp4"

    if not video_path.exists() or not audio_path.exists():
        return JSONResponse(status_code=400, content={"error": "Input files not found"})

    # Run Wav2Lip inference
    command = [
    sys.executable, "inference.py",
    "--checkpoint_path", "checkpoints/wav2lip_gan.pth",
    "--face", str(video_path),
    "--audio", str(audio_path),
    ]

    try:
        result = subprocess.run(
        command,
        cwd=str(BASE_DIR),
        check=True,
        capture_output=True,
        text=True
    )
    except subprocess.CalledProcessError as e:
        return JSONResponse(status_code=500, content={"error": f"Processing failed: {str(e)}"})

    if output_path.exists():
        return {"message": "Processing completed successfully", "output": str(output_path)}
    else:
        return JSONResponse(status_code=500, content={"error": "Output file not generated"})


@app.get("/download")
def download_result():
    """
    Endpoint to download the processed video file.
    """
    output_path = OUTPUT_DIR / "result_voice.mp4"
    if output_path.exists():
        return FileResponse(output_path, media_type="video/mp4", filename="result_voice.mp4")
    else:
        return JSONResponse(status_code=404, content={"error": "Result file not found"})


@app.delete("/cleanup")
def cleanup():
    """
    Delete all input and output files to reset the environment.
    """
    for file in INPUT_DIR.iterdir():
        file.unlink()
    for file in OUTPUT_DIR.iterdir():
        file.unlink()
    return {"message": "All input and output files deleted"}


# Start ngrok and run FastAPI
if __name__ == "__main__":
    # Start ngrok tunnel
    ngrok_tunnel = ngrok.connect(8000, bind_tls=True)
    print("Public URL:", ngrok_tunnel.public_url)

    # Prevent asyncio issues with Colab
    nest_asyncio.apply()

    # Start FastAPI server
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [2348]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://778d-34-19-32-88.ngrok-free.app
Received video file: input_video.mp4
Received audio file: input_audio.wav
Video file content type: video/mp4
Audio file content type: audio/wav
INFO:     2400:adc1:120:3300:d440:4801:2d37:7015:0 - "POST /upload HTTP/1.1" 200 OK
INFO:     2400:adc1:120:3300:d440:4801:2d37:7015:0 - "POST /process?video_filename=/content/sample_data/input_video.mp4&audio_filename=/content/sample_data/input_audio.wav HTTP/1.1" 200 OK
INFO:     2400:adc1:120:3300:d440:4801:2d37:7015:0 - "OPTIONS /download HTTP/1.1" 200 OK
INFO:     2400:adc1:120:3300:d440:4801:2d37:7015:0 - "GET /download HTTP/1.1" 200 OK
